In [156]:
#Получение множества участников соревнования
def get_users_list(df):
    setik = set()
    for i in range(len(df)):
        setik.add(df.loc[i, 'id'])
    return setik
#Получить множества, кто выбыл, а кто остался
def get_good_and_bad(df1, df2):
    good = set() # множество тех, кто останется
    bad = set() # множество тех, кто бросил курс
    users1 = get_users_list(df1)
    users2 = get_users_list(df2)
    for user in users1:
        if user in users2:
            good.add(user)
        else:
            bad.add(user)
    return good, bad

def create_df_with_target(df1, df2):
    good, bad = get_good_and_bad(df1, df2)
    df1['not_leaved'] = df1['id'].isin(good).astype(int)
    return df1

In [157]:
#Предсказывание поведения, на последующих данных, обучая модель на прошлых данных.
#Оценка на Метрики + Предсказание кол-ва людей.
#Работает с весами классов, на precision и recall
from catboost import CatBoostClassifier, Pool, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Эта функция для предсказания, когда есть rate
def study_CB2_model(df1, df2, target):
    X_train = df1.drop(target, axis=1)
    X_train = X_train.drop('id', axis=1)
    y_train = df1[target]
    X_test = df2.drop(target, axis=1)
    X_test = X_test.drop('id', axis=1)
    y_test = df2[target]
    categorical_features_indices = []
    
    train_pool = Pool(
        data=X_train,
        label=y_train,
        cat_features=categorical_features_indices
    )

    test_pool = Pool(
        data=X_test,
        label=y_test,
        cat_features=categorical_features_indices
    )
    
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.1,
        depth=3,
        l2_leaf_reg=3,
        loss_function='Logloss',
        random_seed=42,
        verbose=False,
        eval_metric=metrics.F1(),
        class_weights=[1.15 , 1]
    )
    model.fit(train_pool, eval_set=test_pool, verbose=False)
    y_pred = model.predict(test_pool)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label=0)
    recall = recall_score(y_test, y_pred, pos_label=0)
    f1 = f1_score(y_test, y_pred, pos_label=0)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    return y_pred, y_test, model

#Это функция для случая, когда нет rate
def study_CB1_model(df1, df2, target):
    X_train = df1.drop(target, axis=1)
    X_train = X_train.drop('rate', axis=1)
    X_train = X_train.drop('id', axis=1)
    y_train = df1[target]
    X_test = df2.drop(target, axis=1)
    X_test = X_test.drop('rate', axis=1)
    X_test = X_test.drop('id', axis=1)
    y_test = df2[target]
    categorical_features_indices = []
    
    train_pool = Pool(
        data=X_train,
        label=y_train,
        cat_features=categorical_features_indices
    )

    test_pool = Pool(
        data=X_test,
        label=y_test,
        cat_features=categorical_features_indices
    )
    
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.1,
        depth=3,
        l2_leaf_reg=3,
        loss_function='Logloss',
        random_seed=42,
        verbose=False,
        eval_metric=metrics.F1(),
        class_weights=[1.15, 1]
    )
    model.fit(train_pool, eval_set=test_pool, verbose=False)
    y_pred = model.predict(test_pool)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label=0)
    recall = recall_score(y_test, y_pred, pos_label=0)
    f1 = f1_score(y_test, y_pred, pos_label=0)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    return y_pred, y_test, model

In [158]:
#Загрузка данных.
import pandas as pd
import numpy as np
df1 = pd.read_csv('a1.csv')
df2 = pd.read_csv('a2.csv')
df3 = pd.read_csv('a3.csv')
df4 = pd.read_csv('a4.csv')
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = df2.drop('Unnamed: 0', axis=1)
df3 = df3.drop('Unnamed: 0', axis=1)
df4 = df4.drop('Unnamed: 0', axis=1)

#Обработка столбика rate первым способом,
#просто выставляя рейтинг по позиции в таблице
df1 = df1.reset_index()
df1['rate'] = (df1['index'] + 1)/len(df1)
df1 = df1.drop('index', axis=1)
df2 = df2.reset_index()
df2['rate'] = (df2['index'] + 1)/len(df2)
df2 = df2.drop('index', axis=1)
df3 = df3.reset_index()
df3['rate'] = (df3['index'] + 1)/len(df3)
df3 = df3.drop('index', axis=1)
df4 = df4.reset_index()
df4['rate'] = (df4['index'] + 1)/len(df4)
df4 = df4.drop('index', axis=1)

#Получаем 3 таргетных таблички, а так же слитую 1 и 2 для предсказания третьей
df1_target = create_df_with_target(df1, df2)
df2_target = create_df_with_target(df2, df3)
df3_target = create_df_with_target(df3, df4)
df12_target =pd.concat([df1, df2], axis=0)

In [159]:
target = 'not_leaved'
print("Метрики, предсказания оттока с курса после 2-го модуля, обученная по оттоку после 1-го модуля")
y_pred1, y_test1, model0 = study_CB2_model(df1_target, df2_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 1-го модуля")
y_pred2, y_test2, model1 = study_CB2_model(df1_target, df3_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 2-го модуля")
y_pred3, y_test3, model2 = study_CB2_model(df2_target, df3_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обучченная по оттоку после 1-го и 2-го модулей")
y_pred4, y_test4, model0 = study_CB2_model(df12_target, df3_target, target)
print("Не обращать внимания на эти метрики, это просто для сохранения модели обученой по 3 неделе")
k, l, model3 = study_CB2_model(df3_target, df1_target, target)

Метрики, предсказания оттока с курса после 2-го модуля, обученная по оттоку после 1-го модуля
Accuracy: 0.8115
Precision: 0.5385
Recall: 0.6525
F1-score: 0.5900
Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 1-го модуля
Accuracy: 0.7800
Precision: 0.5049
Recall: 0.7243
F1-score: 0.5950
Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 2-го модуля
Accuracy: 0.8509
Precision: 0.6919
Recall: 0.5981
F1-score: 0.6416
Метрики, предсказания оттока с курса после 3-го модуля, обучченная по оттоку после 1-го и 2-го модулей
Accuracy: 0.8332
Precision: 0.6154
Recall: 0.6729
F1-score: 0.6429
Не обращать внимания на эти метрики, это просто для сохранения модели обученой по 3 неделе
Accuracy: 0.7378
Precision: 0.8556
Recall: 0.5385
F1-score: 0.6609


In [160]:
print("1 -> 2 таблица")
#print("Количество человек в табличке: ", len(y_pred1))
#print("Количество предсказанное отток: ", len(y_pred1) - sum(y_pred1)) 
#print("Количестов реальный отток", len(y_pred1) - sum(y_test1))
predict_leave1 = len(y_pred1) - sum(y_pred1)
real_leave1 = len(y_pred1) - sum(y_test1)
percentage1 = (predict_leave1 / (real_leave1 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage1:.1f}")

print("1 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred2))
#print("Количество предсказанное отток: ", len(y_pred2) - sum(y_pred2)) 
#print("Количестов реальный отток", len(y_pred2) - sum(y_test2))
predict_leave2 = len(y_pred2) - sum(y_pred2)
real_leave2 = len(y_pred2) - sum(y_test2)
percentage2 = (predict_leave2 / (real_leave2 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage2:.1f}")

print("2 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred3))
#print("Количество предсказанное отток: ", len(y_pred3) - sum(y_pred3)) 
#print("Количестов реальный отток", len(y_pred3) - sum(y_test3))
predict_leave3 = len(y_pred3) - sum(y_pred3)
real_leave3 = len(y_pred3) - sum(y_test3)
percentage3 = (predict_leave3 / (real_leave3 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage3:.1f}")

print("12 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred4))
#print("Количество предсказанное отток: ", len(y_pred4) - sum(y_pred4)) 
#print("Количестов реальный отток", len(y_pred4) - sum(y_test4))
predict_leave4 = len(y_pred4) - sum(y_pred4)
real_leave4 = len(y_pred4) - sum(y_test4)
percentage4 = (predict_leave4 / (real_leave4 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage4:.1f}")

1 -> 2 таблица
Процент отклонения от реальности: 21.2
1 -> 3 таблица
Процент отклонения от реальности: 43.5
2 -> 3 таблица
Процент отклонения от реальности: -13.6
12 -> 3 таблица
Процент отклонения от реальности: 9.3


In [161]:
#перезагрузка данных.
import pandas as pd
import numpy as np
df1 = pd.read_csv('a1.csv')
df2 = pd.read_csv('a2.csv')
df3 = pd.read_csv('a3.csv')
df4 = pd.read_csv('a4.csv')
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = df2.drop('Unnamed: 0', axis=1)
df3 = df3.drop('Unnamed: 0', axis=1)
df4 = df4.drop('Unnamed: 0', axis=1)

#Вариант, если брать рейтинг для участников
#с одинаковым положением по верхней границе диапазона.
def floatize_rate(rate):
    try:
        rate = int(rate)
    except:
        k = 0
        while (rate[k] != '-'):
            k += 1
        rate = int(rate[:k])
    return rate
df1['rate'] = df1['rate'].apply(floatize_rate) 
df2['rate'] = df2['rate'].apply(floatize_rate) 
df3['rate'] = df3['rate'].apply(floatize_rate) 
df4['rate'] = df4['rate'].apply(floatize_rate)
df1['rate'] = df1['rate']/len(df1)
df2['rate'] = df2['rate']/len(df1)
df3['rate'] = df3['rate']/len(df1)
df4['rate'] = df4['rate']/len(df1)

#Получаем 3 таргетных таблички, а так же слитую 1 и 2 для предсказания третьей
df1_target = create_df_with_target(df1, df2)
df2_target = create_df_with_target(df2, df3)
df3_target = create_df_with_target(df3, df4)
df12_target =pd.concat([df1, df2], axis=0)

In [162]:
target = 'not_leaved'
print("Метрики, предсказания оттока с курса после 2-го модуля, обученная по оттоку после 1-го модуля")
y_pred1, y_test1, model0 = study_CB2_model(df1_target, df2_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 1-го модуля")
y_pred2, y_test2, model4 = study_CB2_model(df1_target, df3_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 2-го модуля")
y_pred3, y_test3, model5 = study_CB2_model(df2_target, df3_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обучченная по оттоку после 1-го и 2-го модулей")
y_pred4, y_test4, model0 = study_CB2_model(df12_target, df3_target, target)
print("Не обращать внимания на эти метрики, это просто для сохранения модели обученой по 3 неделе")
k, l, model6 = study_CB2_model(df3_target, df1_target, target)

Метрики, предсказания оттока с курса после 2-го модуля, обученная по оттоку после 1-го модуля
Accuracy: 0.8203
Precision: 0.5548
Recall: 0.6864
F1-score: 0.6136
Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 1-го модуля
Accuracy: 0.8425
Precision: 0.6703
Recall: 0.5794
F1-score: 0.6216
Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 2-го модуля
Accuracy: 0.8530
Precision: 0.7110
Recall: 0.5748
F1-score: 0.6357
Метрики, предсказания оттока с курса после 3-го модуля, обучченная по оттоку после 1-го и 2-го модулей
Accuracy: 0.8498
Precision: 0.7011
Recall: 0.5701
F1-score: 0.6289
Не обращать внимания на эти метрики, это просто для сохранения модели обученой по 3 неделе
Accuracy: 0.7444
Precision: 0.8226
Recall: 0.5884
F1-score: 0.6861


In [163]:
print("1 -> 2 таблица")
#print("Количество человек в табличке: ", len(y_pred1))
#print("Количество предсказанное отток: ", len(y_pred1) - sum(y_pred1)) 
#print("Количестов реальный отток", len(y_pred1) - sum(y_test1))
predict_leave1 = len(y_pred1) - sum(y_pred1)
real_leave1 = len(y_pred1) - sum(y_test1)
percentage1 = (predict_leave1 / (real_leave1 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage1:.1f}")

print("1 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred2))
#print("Количество предсказанное отток: ", len(y_pred2) - sum(y_pred2)) 
#print("Количестов реальный отток", len(y_pred2) - sum(y_test2))
predict_leave2 = len(y_pred2) - sum(y_pred2)
real_leave2 = len(y_pred2) - sum(y_test2)
percentage2 = (predict_leave2 / (real_leave2 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage2:.1f}")

print("2 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred3))
#print("Количество предсказанное отток: ", len(y_pred3) - sum(y_pred3)) 
#print("Количестов реальный отток", len(y_pred3) - sum(y_test3))
predict_leave3 = len(y_pred3) - sum(y_pred3)
real_leave3 = len(y_pred3) - sum(y_test3)
percentage3 = (predict_leave3 / (real_leave3 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage3:.1f}")

print("12 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred4))
#print("Количество предсказанное отток: ", len(y_pred4) - sum(y_pred4)) 
#print("Количестов реальный отток", len(y_pred4) - sum(y_test4))
predict_leave4 = len(y_pred4) - sum(y_pred4)
real_leave4 = len(y_pred4) - sum(y_test4)
percentage4 = (predict_leave4 / (real_leave4 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage4:.1f}")

1 -> 2 таблица
Процент отклонения от реальности: 23.7
1 -> 3 таблица
Процент отклонения от реальности: -13.6
2 -> 3 таблица
Процент отклонения от реальности: -19.2
12 -> 3 таблица
Процент отклонения от реальности: -18.7


In [164]:
#Загрузка данных.
import pandas as pd
import numpy as np
df1 = pd.read_csv('a1.csv')
df2 = pd.read_csv('a2.csv')
df3 = pd.read_csv('a3.csv')
df4 = pd.read_csv('a4.csv')
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = df2.drop('Unnamed: 0', axis=1)
df3 = df3.drop('Unnamed: 0', axis=1)
df4 = df4.drop('Unnamed: 0', axis=1)

#Получаем 3 таргетных таблички, а так же слитую 1 и 2 для предсказания третьей
df1_target = create_df_with_target(df1, df2)
df2_target = create_df_with_target(df2, df3)
df3_target = create_df_with_target(df3, df4)
df12_target =pd.concat([df1, df2], axis=0)

In [165]:
target = 'not_leaved'
print("Метрики, предсказания оттока с курса после 2-го модуля, обученная по оттоку после 1-го модуля")
y_pred1, y_test1, model0 = study_CB1_model(df1_target, df2_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 1-го модуля")
y_pred2, y_test2, model7 = study_CB1_model(df1_target, df3_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 2-го модуля")
y_pred3, y_test3, model8 = study_CB1_model(df2_target, df3_target, target)
print("Метрики, предсказания оттока с курса после 3-го модуля, обучченная по оттоку после 1-го и 2-го модулей")
y_pred4, y_test4, model0 = study_CB1_model(df12_target, df3_target, target)
print("Не обращать внимания на эти метрики, это просто для сохранения модели обученой по 3 неделе")
k, l, model9 = study_CB1_model(df3_target, df1_target, target)

Метрики, предсказания оттока с курса после 2-го модуля, обученная по оттоку после 1-го модуля
Accuracy: 0.8132
Precision: 0.5408
Recall: 0.6737
F1-score: 0.6000
Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 1-го модуля
Accuracy: 0.8060
Precision: 0.5490
Recall: 0.7336
F1-score: 0.6280
Метрики, предсказания оттока с курса после 3-го модуля, обученная по оттоку после 2-го модуля
Accuracy: 0.8498
Precision: 0.7134
Recall: 0.5467
F1-score: 0.6190
Метрики, предсказания оттока с курса после 3-го модуля, обучченная по оттоку после 1-го и 2-го модулей
Accuracy: 0.8384
Precision: 0.6300
Recall: 0.6682
F1-score: 0.6485
Не обращать внимания на эти метрики, это просто для сохранения модели обученой по 3 неделе
Accuracy: 0.7425
Precision: 0.8358
Recall: 0.5694
F1-score: 0.6774


In [166]:
print("1 -> 2 таблица")
#print("Количество человек в табличке: ", len(y_pred1))
predict_leave1 = len(y_pred1) - sum(y_pred1)
real_leave1 = len(y_pred1) - sum(y_test1)
percentage1 = (predict_leave1 / (real_leave1 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage1:.1f}")

print("1 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred2))
predict_leave2 = len(y_pred2) - sum(y_pred2)
real_leave2 = len(y_pred2) - sum(y_test2)
percentage2 = (predict_leave2 / (real_leave2 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage2:.1f}")

print("2 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred3))
predict_leave3 = len(y_pred3) - sum(y_pred3)
real_leave3 = len(y_pred3) - sum(y_test3)
percentage3 = (predict_leave3 / (real_leave3 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage3:.1f}")

print("2 -> 3 таблица")
#print("Количество человек в табличке: ", len(y_pred4))
predict_leave4 = len(y_pred4) - sum(y_pred4)
real_leave4 = len(y_pred4) - sum(y_test4)
percentage4 = (predict_leave4 / (real_leave4 / 100)) - 100
print(f"Процент отклонения от реальности: {percentage4:.1f}")

1 -> 2 таблица
Процент отклонения от реальности: 24.6
1 -> 3 таблица
Процент отклонения от реальности: 33.6
2 -> 3 таблица
Процент отклонения от реальности: -23.4
2 -> 3 таблица
Процент отклонения от реальности: 6.1


In [167]:
model1.save_model("models/1.cbm")
model2.save_model("models/2.cbm")
model3.save_model("models/3.cbm")

model4.save_model("models/4.cbm")
model5.save_model("models/5.cbm")
model6.save_model("models/6.cbm")

model7.save_model("models/7.cbm")
model8.save_model("models/8.cbm")
model9.save_model("models/9.cbm")